In [1]:
pip install pandas datasets unsloth torch transformers requests cupy-cuda12x


  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl (1.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1+cu121, but you have torch 2.6.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
import requests
import json

# DeepSeek API setup
DEEPSEEK_API_KEY = "sk-732ea226899242339e5d25944abbafd7"
DEEPSEEK_URL = "https://api.deepseek.com/v1/chat/completions"

# Load dataset from Hugging Face
dataset = load_dataset("medalpaca/medical_meadow_medqa")
df = dataset["train"].to_pandas()

# Verify column names
print(df.columns)  # Debugging step

# Correct column names
HEALTH_TOPICS = ["neonatal", "pregnancy", "obgyn", "baby", "birth", "fetal", "labor", "ectopic", "preeclampsia", "gestation"]
filtered_df = df[df["input"].str.lower().apply(lambda x: any(topic in x for topic in HEALTH_TOPICS))]

# Generate synthetic data using DeepSeek API
def generate_synthetic_qa(prompt):
    payload = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 200
    }
    headers = {"Authorization": f"Bearer {DEEPSEEK_API_KEY}", "Content-Type": "application/json"}
    response = requests.post(DEEPSEEK_URL, json=payload, headers=headers).json()

    print(response)  # Debug API response

    if "choices" in response:
        return response["choices"][0]["message"]["content"]
    else:
        return "Error: Invalid API Response"

synthetic_data = []
prompts = [
    "Generate a question and answer about neonatal jaundice treatment.",
    "Generate a Q&A pair about managing preeclampsia in pregnancy.",
    "Generate a question and answer about postpartum hemorrhage."
]

for prompt in prompts:
    response = generate_synthetic_qa(prompt)
    lines = response.split("\n")
    if len(lines) >= 2:
        q, a = lines[0], lines[1]
        synthetic_data.append({"input": q.strip(), "output": a.strip()})

# Combine datasets
synthetic_df = pd.DataFrame(synthetic_data)
combined_df = pd.concat([filtered_df[["input", "output"]], synthetic_df], ignore_index=True)

# Save to JSON
combined_df.to_json("health_data.json", orient="records")
dataset = Dataset.from_pandas(combined_df)
print(f"Total entries: {len(combined_df)}")


Index(['input', 'instruction', 'output'], dtype='object')
{'id': '25f54daf-eadb-4eed-a024-8e5c568723d1', 'object': 'chat.completion', 'created': 1743187043, 'model': 'deepseek-chat', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "**Question:** What are the common treatment options for neonatal jaundice?  \n\n**Answer:** The most common treatments for neonatal jaundice include:  \n1. **Phototherapy** – Exposure to special blue-spectrum lights that help break down bilirubin in the baby's skin.  \n2. **Frequent Feeding** – Increasing breast milk or formula intake helps promote bilirubin excretion through stool.  \n3. **Intravenous Immunoglobulin (IVIG)** – Used in cases of severe jaundice caused by blood group incompatibility (e.g., Rh or ABO incompatibility).  \n4. **Exchange Transfusion** – A rare but critical procedure for extreme cases where bilirubin levels are dangerously high.  \n\nTreatment choice depends on bilirubin levels, the baby's age, and underlying c

In [4]:
import pandas as pd
from datasets import load_dataset, Dataset
import requests
import json
from unsloth import FastLanguageModel

# DeepSeek API setup
DEEPSEEK_API_KEY = "sk-732ea226899242339e5d25944abbafd7"
DEEPSEEK_URL = "https://api.deepseek.com/v1/chat/completions"

# Load dataset from Hugging Face
dataset = load_dataset("medalpaca/medical_meadow_medqa")
df = dataset["train"].to_pandas()

# Verify column names
print("Columns in dataset:", df.columns)

# Define relevant health topics
HEALTH_TOPICS = ["neonatal", "pregnancy", "obgyn", "baby", "birth", "fetal", "labor", "ectopic", "preeclampsia", "gestation"]

# Filter for relevant topics (CPU-based)
filtered_df = df[df["input"].str.lower().apply(lambda x: any(topic in x for topic in HEALTH_TOPICS))]
print(f"Filtered entries: {len(filtered_df)}")

# Generate synthetic data using DeepSeek API
def generate_synthetic_qa(prompt):
    payload = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 200
    }
    headers = {"Authorization": f"Bearer {DEEPSEEK_API_KEY}", "Content-Type": "application/json"}
    response = requests.post(DEEPSEEK_URL, json=payload, headers=headers).json()

    print("API Response:", response)

    if "choices" in response and response["choices"]:
        content = response["choices"][0]["message"]["content"]
        lines = [line.strip() for line in content.split("\n") if line.strip()]
        if len(lines) >= 2:
            return lines[0], lines[1]
        else:
            return "Error: Unexpected format", content
    else:
        return "Error: Invalid API Response", "No content"

synthetic_data = []
prompts = [
    "Generate a question and answer about neonatal jaundice treatment.",
    "Generate a Q&A pair about managing preeclampsia in pregnancy.",
    "Generate a question and answer about postpartum hemorrhage."
]

for prompt in prompts:
    question, answer = generate_synthetic_qa(prompt)
    synthetic_data.append({"input": question, "output": answer})

# Combine datasets
synthetic_df = pd.DataFrame(synthetic_data)
combined_df = pd.concat([filtered_df[["input", "output"]], synthetic_df], ignore_index=True)

# Save to JSON
combined_df.to_json("health_data.json", orient="records")
dataset = Dataset.from_pandas(combined_df)
print(f"Total entries: {len(combined_df)}")

# Fine-tuning with unsloth
print("Starting fine-tuning...")
model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B"  # Base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,  # Auto-detect
    load_in_4bit=True  # For efficiency
)

# Load the dataset
dataset = load_dataset("json", data_files="health_data.json")["train"]

# Define prompt template
prompt_template = """### Instruction: You are Sharma, a medical expert specializing in neonatal, pregnancy, and OB-GYN cases. Respond only to health queries in these areas. For non-relevant queries, say: "I’m specialized in neonatal, pregnancy, and OB-GYN cases only. Please ask a related question."
Question: {input}
Answer: {output}"""

def format_prompt(example):
    return {"text": prompt_template.format(input=example["input"], output=example["output"])}

dataset = dataset.map(format_prompt)

# Fine-tune with LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0
)

trainer = FastLanguageModel.Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args={
        "per_device_train_batch_size": 2,
        "gradient_accumulation_steps": 4,
        "max_steps": 500,
        "learning_rate": 2e-4,
        "fp16": True,
        "logging_steps": 10
    }
)

trainer.train()

# Save the fine-tuned model
model.save_pretrained("sharma-health-model")
tokenizer.save_pretrained("sharma-health-model")
print("Model saved to 'sharma-health-model'")

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [6]:
import pandas as pd
from datasets import load_dataset, Dataset
import requests
import json
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

# DeepSeek API setup
DEEPSEEK_API_KEY = "sk-732ea226899242339e5d25944abbafd7"
DEEPSEEK_URL = "https://api.deepseek.com/v1/chat/completions"

# Load dataset from Hugging Face
dataset = load_dataset("medalpaca/medical_meadow_medqa")
df = dataset["train"].to_pandas()

# Verify column names
print("Columns in dataset:", df.columns)

# Define relevant health topics
HEALTH_TOPICS = ["neonatal", "pregnancy", "obgyn", "baby", "birth", "fetal", "labor", "ectopic", "preeclampsia", "gestation"]

# Filter for relevant topics
filtered_df = df[df["input"].str.lower().apply(lambda x: any(topic in x for topic in HEALTH_TOPICS))]
print(f"Filtered entries: {len(filtered_df)}")

# Generate synthetic data using DeepSeek API
def generate_synthetic_qa(prompt):
    payload = {
        "model": "deepseek-chat",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 200
    }
    headers = {"Authorization": f"Bearer {DEEPSEEK_API_KEY}", "Content-Type": "application/json"}
    response = requests.post(DEEPSEEK_URL, json=payload, headers=headers).json()
    print("API Response:", response)
    if "choices" in response and response["choices"]:
        content = response["choices"][0]["message"]["content"]
        lines = [line.strip() for line in content.split("\n") if line.strip()]
        if len(lines) >= 2:
            return lines[0], lines[1]
        else:
            return "Error: Unexpected format", content
    else:
        return "Error: Invalid API Response", "No content"

synthetic_data = []
prompts = [
    "Generate a question and answer about neonatal jaundice treatment.",
    "Generate a Q&A pair about managing preeclampsia in pregnancy.",
    "Generate a question and answer about postpartum hemorrhage."
]

for prompt in prompts:
    question, answer = generate_synthetic_qa(prompt)
    synthetic_data.append({"input": question, "output": answer})

# Combine datasets
synthetic_df = pd.DataFrame(synthetic_data)
combined_df = pd.concat([filtered_df[["input", "output"]], synthetic_df], ignore_index=True)

# Save to JSON
combined_df.to_json("health_data.json", orient="records")
dataset = Dataset.from_pandas(combined_df)
print(f"Total entries: {len(combined_df)}")

# Fine-tuning with transformers
print("Starting fine-tuning...")
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token

# Prepare dataset
prompt_template = """### Instruction: You are Sharma, a medical expert specializing in neonatal, pregnancy, and OB-GYN cases. Respond only to health queries in these areas. For non-relevant queries, say: "I’m specialized in neonatal, pregnancy, and OB-GYN cases only. Please ask a related question."
Question: {input}
Answer: {output}"""

def format_prompt(example):
    return prompt_template.format(input=example["input"], output=example["output"])

dataset = dataset.map(lambda x: {"text": format_prompt(x)})

# Tokenize and prepare labels
def tokenize_function(examples):
    encodings = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # Labels are the same as input_ids for causal LM
    return encodings

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text", "input", "output"])  # Remove unnecessary columns
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Split dataset
train_dataset = tokenized_dataset.shuffle(seed=42).select(range(int(len(tokenized_dataset) * 0.9)))
eval_dataset = tokenized_dataset.shuffle(seed=42).select(range(int(len(tokenized_dataset) * 0.9), len(tokenized_dataset)))

# Training arguments
training_args = TrainingArguments(
    output_dir="./sharma-health-model",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=2e-5,
    fp16=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("sharma-health-model")
tokenizer.save_pretrained("sharma-health-model")
print("Model saved to 'sharma-health-model'")

Columns in dataset: Index(['input', 'instruction', 'output'], dtype='object')
Filtered entries: 2682
API Response: {'id': '6acac08f-ca57-4f80-bbc1-0ff80cafb31f', 'object': 'chat.completion', 'created': 1743199914, 'model': 'deepseek-chat', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "**Question:**  \nWhat is the most common and effective treatment for neonatal jaundice caused by elevated bilirubin levels?  \n\n**Answer:**  \nThe most common and effective treatment for neonatal jaundice is **phototherapy**, which uses special blue-spectrum light to break down bilirubin in the baby's skin, making it easier for the body to eliminate. In severe cases, an **exchange transfusion** may be required to rapidly lower bilirubin levels. Treatment decisions are based on the infant's age, bilirubin levels, and overall health. Early feeding (breastfeeding or formula) also helps by promoting bilirubin excretion through stool.  \n\nWould you like additional details on treatment

Map:   0%|          | 0/2685 [00:00<?, ? examples/s]

Map:   0%|          | 0/2685 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,4.110200
20,2.445700
30,2.316300
40,2.079200
50,1.903600
60,1.711900
70,1.698400
80,1.627300
90,1.518900
100,1.540100


Model saved to 'sharma-health-model'
